In [1]:
import anndata
import matplotlib.pyplot as plt
import numpy as np
import scanpy as sc
import scgen
from scipy.stats import spearmanr
from scvi.data import cortex, smfish
from scvi.external import GIMVI
import sys
import os
import pandas as pd
from multiprocessing import Pool
from adpbulk import ADPBulk
import gc
import seaborn as sns
from torchviz import make_dot

import scvi
scvi.settings.num_threads = 10

seed = 42
np.random.seed(seed)



/scratch/alper.eroglu/miniconda3/lib/python3.10/site-packages/anndata/utils.py:434: FutureWarning: Importing read_csv from `anndata` is deprecated. Import anndata.io.read_csv instead.
  warnings.warn(msg, FutureWarning)
/scratch/alper.eroglu/miniconda3/lib/python3.10/site-packages/anndata/utils.py:434: FutureWarning: Importing read_excel from `anndata` is deprecated. Import anndata.io.read_excel instead.
  warnings.warn(msg, FutureWarning)
/scratch/alper.eroglu/miniconda3/lib/python3.10/site-packages/anndata/utils.py:434: FutureWarning: Importing read_hdf from `anndata` is deprecated. Import anndata.io.read_hdf instead.
  warnings.warn(msg, FutureWarning)
/scratch/alper.eroglu/miniconda3/lib/python3.10/site-packages/anndata/utils.py:434: FutureWarning: Importing read_loom from `anndata` is deprecated. Import anndata.io.read_loom instead.
  warnings.warn(msg, FutureWarning)
/scratch/alper.eroglu/miniconda3/lib/python3.10/site-packages/anndata/utils.py:434: FutureWarning: Importing read_

In [8]:
gsell_adata = sc.read_h5ad("/scratch/alper.eroglu/cirrus/data/Gsell_BC_Coh4.h5ad")
metadata = pd.read_csv("/scratch/alper.eroglu/Coh4_Human_Mice/coh4_metadata.csv", index_col = 0)
gsell_adata.obs = gsell_adata.obs.merge(metadata,  left_index=True, right_index=True)
gsell_adata = gsell_adata[~gsell_adata.obs["cellType"].isna()]

adpb = ADPBulk(gsell_adata, ["orig.ident", "cellType"])
alphas = adpb.fit_transform()
# alphas.index = alphas.index.map(lambda x: x.split(".")[1])

alphas.shape



Aggregating Samples: 100%|██████████| 80/80 [00:24<00:00,  3.29it/s]


(73, 16978)

In [9]:
sample_meta = adpb.get_meta()

sample_meta.head()


,cellType,orig.ident,SampleName
0,Bcells,629-1,cellType.Bcells-orig.ident.629-1
1,Bcells,629-5,cellType.Bcells-orig.ident.629-5
2,Bcells,632-2,cellType.Bcells-orig.ident.632-2
3,Bcells,634-1,cellType.Bcells-orig.ident.634-1
4,Bcells,634-5,cellType.Bcells-orig.ident.634-5


In [10]:
alphas.index

Index(['cellType.Bcells-orig.ident.629-1', 'cellType.Bcells-orig.ident.629-5',
       'cellType.Bcells-orig.ident.632-2', 'cellType.Bcells-orig.ident.634-1',
       'cellType.Bcells-orig.ident.634-5', 'cellType.Bcells-orig.ident.636-1',
       'cellType.Bcells-orig.ident.636-4',
       'cellType.CD4Tcells-orig.ident.629-1',
       'cellType.CD4Tcells-orig.ident.629-5',
       'cellType.CD4Tcells-orig.ident.632-3',
       'cellType.CD4Tcells-orig.ident.634-1',
       'cellType.CD4Tcells-orig.ident.634-5',
       'cellType.CD4Tcells-orig.ident.636-1',
       'cellType.CD4Tcells-orig.ident.636-4',
       'cellType.CD8Tcells-orig.ident.629-1',
       'cellType.CD8Tcells-orig.ident.629-5',
       'cellType.CD8Tcells-orig.ident.632-2',
       'cellType.CD8Tcells-orig.ident.632-3',
       'cellType.CD8Tcells-orig.ident.634-1',
       'cellType.CD8Tcells-orig.ident.634-5',
       'cellType.CD8Tcells-orig.ident.636-1',
       'cellType.CD8Tcells-orig.ident.636-4', 'cellType.DCs-orig.ident.629-1

In [13]:
alphas.to_csv("/scratch/alper.eroglu/data/BC_pseudobulk_counts.csv")

In [14]:
sample_meta.to_csv("/scratch/alper.eroglu/data/BC_pseudobulk_metadata.csv")

In [ ]:
del